In [11]:
from functools import lru_cache
import numpy as np
from sympy.utilities import lambdify
from numba import prange
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import inv
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sympy as sm
from time import time
from scipy.sparse import *
import pandas as pd
import time
import math

In [12]:
class Equation():
    #Set parameters 
    ld = .005
    k = .2
    beta = .5
    b = .5
    end_time = 1
    x0 = 10
            
    t = sm.symbols("t")
    #Set boundary conditions
    g_0 = ld - 2*k*b*(1/sm.tanh(k*x0 - ld*t))
    g_1 = ld - 2*k*b*(1/sm.tanh(k*(1 + x0) - ld*t))
    h_0 = -2*k*ld*b*(1/sm.sinh(k*x0 - ld*t))**2
    h_1 = -2*k*ld*b*(1/sm.sinh(k*(1 + x0) - ld*t))**2
    ############################################################
    dg_0 = sm.apart((g_0.diff(t)))
    dg_1 = sm.apart((g_1.diff(t)))
    dh_0 = h_0.diff(t)
    dh_1 = h_1.diff(t)
    g_0 = lambdify(t, g_0)
    g_1 = lambdify(t, g_1)
    h_0 = lambdify(t, h_0)
    h_1 = lambdify(t, h_1)
    dg_0 = lambdify(t, dg_0)
    dg_1 = lambdify(t, dg_1)
    dh_0 = lambdify(t, dh_0)
    dh_1 = lambdify(t, dh_1)

    #Set initial conditions
    @staticmethod
    def u0(x):return Equation.ld - 2*Equation.k*Equation.b*(1/np.tanh(Equation.k*(Equation.x0 + x)))
    
    def v0(x):return -2*Equation.k*Equation.ld*Equation.b*(1/np.sinh(Equation.k*(Equation.x0 + x)))**2


    #Exact Solution
    def u(x) : Equation.ld - 2*Equation.k*Equation.b*(1/np.tanh(k*(Equation.x0 + x) - Equation.ld * Equation.end_time))
    def v(x):   -2*Equation.k*Equation.ld*Equation.b*(1/np.sinh(k*(Equation.x0 + x)- Equation.ld * Equation.end_time))**2

    errors_u = []
    errors_v = []

    #Pandas datasheet
    u_solver = {}
    v_solver = {}
    u_exact  = {}
    v_exact = {}

    #Graph comps
    u_graph_calc = []
    u_graph_exact = []
    v_graph_calc = []
    v_graph_exact = []

    def __init__(self, Ns):

        #Set parameters 
        self.Ns = Ns
        for i in self.Ns:
            self.solve(i)
        
        #Graphs
        self.graph_u = self.graph(self.u_graph_calc, self.u_graph_exact)
        self.graph_v = self.graph(self.v_graph_calc, self.v_graph_exact)
                
    
    def error_compute(self, a, e):
        return  np.linalg.norm(e  -  a, 2)
        

    def solve(self, N):

        eq = Solver(N)
        u_solver[N], v_solver[N] = eq.algo_1() 
        u_exact[N], v_exact[N] = np.array(list(map(lambda x:self.u(x, self.end_time), eq.domain))),\
                                             np.array(list(map(lambda x:self.v(x, self.end_time), eq.domain))) 
        
        errors_u[N], self.errors_v[N]= self.error_compute(self.u_solver[N], self.u_exact[N]),\
                self.error_compute(self.v_solver[N], self.v_exact[N])
        
        u_graph_calc.append(go.Scatter( y=u_solver[N] ,\
                                  visible=False, x=eq.domain, line={'color': 'red'},\
                                  name=f"Approximate {i}"))
        u_graph_exact.append(go.Scatter( y=u_solver[N],\
                                 visible=False,x=eq.domain, line={'color': 'blue'},\
                                 name=f"Exact {i}"))    
        v_graph_calc.append(go.Scatter( y=v_solver[N] ,\
                                  visible=False, x=eq.domain, line={'color': 'red'},\
                                  name=f"Approximate {i}"))
        v_graph_exact.append(go.Scatter( y=v_solver[N],\
                                 visible=False,x=eq.domain, line={'color': 'blue'},\
                                 name=f"Exact {i}"))    
        
    def pandas_u(self, ):
        self.df_u = pd.DataFrame({"Number of Basis Elements":Ns, \
                                   "Approximate":u_solver, "Exact":u_exact}) 
    
    def pandas_v(self, ):
        self.df_v = pd.DataFrame({"Number of Basis Elements":self.Ns, \
                                  "Approximate":v_solver, "Exact":v_exact})
    
    def graph(self, a, e):
        fig = go.Figure(data = a+ e)
        steps = []
        for i in range(len(Ns)):
            # Hide all traces
            step = dict(
                method = 'restyle',  
                args = ['visible', [False] * len(fig.data)],
            )
            # Enable the two traces we want to see
            step['args'][1][i] = True
            step['args'][1][i+len(Ns)] = True

            # Add step to step list
            steps.append(step)

        sliders = [dict(
            steps = steps,
            currentvalue={"prefix": "Basis :  "},

        )]

        fig.layout.sliders = sliders

        iplot(fig, show_link=False)
    

In [13]:
class Solver(Equation):
    
    def __init__(self, N):
        # Parameters
        self.N = N
        self.H = 1/self.N 
        self.shape = (self.N - 1, self.N - 1)
        #Domain
        self.domain = np.array(np.linspace(0, 1, self.N + 1)[1:-1])
        #Matrices
        self.A_inv = inv(self.A_mat())
        
        self.B_left = self.B_left_mat()
        self.B_right =  self.B_right_mat()
        self.B_top = self.B_top_mat()
        self.B_bottom = self.B_bottom_mat()
        self.C = self.C_mat()
        self.D = self.D_mat()
        
         

    def A_mat(self, ):
        diagonals = [[1/6]*(self.N-2), [2/3]*(self.N-1), [1/6]*(self.N-2)]
        return self.H * csc_matrix(diags(diagonals, [-1, 0, 1]))


    def B(self, xi, eta):
       
        
        def Bj(j):
            j -= 1
            if j == 0:
                row = np.array([1, 0, 1])
                col = np.array([0, 1, 1])
                data = np.array([1/3, -1/6, 1/6])
                return csr_matrix((data, (row, col)), shape=self.shape)

            elif j == self.N-2:
                row = np.array([self.N - 1-2, self.N - 1-2, self.N - 1-1])
                col = np.array([self.N - 1-1, self.N - 1-2, self.N - 1-2])
                data = np.array([-1/3, -1/6, 1/6])
                return csr_matrix((data, (row, col)), shape=self.shape)

            else:
                row = np.array([j-1, j, j-1, j+1, j, j+1])
                col = np.array([j-1, j-1, j, j, j+1, j+1])
                data = np.array([-1/6, 1/6, -1/3, 1/3, -1/6, 1/6])
                return csr_matrix((data, (row, col)), shape=self.shape)
            
        data = np.array([xi.dot(Bj(j).dot( eta)) for j in range(1, self.N)])
        row = np.array([0 for j in range(1, self.N) ])
        col = np.array([j -1 for j in range(1, self.N) ])
        return  csc_matrix((data, (row, col)), shape=(1, self.shape[0])).toarray().ravel()


    def B_top_mat(self, ):
        row = np.array([0])
        col = np.array([0])
        data = np.array([-1/3])
        return csc_matrix((data, (row, col)), shape=self.shape)


    def B_bottom_mat(self, ):
        row = np.array([self.N - 1-1])
        col = np.array([self.N - 1-1])
        data = np.array([1/3])
        return csc_matrix((data, (row, col)), shape=self.shape)


    def B_left_mat(self, ):
        row = np.array([0])
        col = np.array([0])
        data = np.array([1/6])
        return csc_matrix((data, (row, col)), shape=self.shape)


    def B_right_mat(self, ):
        row = np.array([self.N - 1-1])
        col = np.array([self.N - 1-1])
        data = np.array([-1/6])
        return csc_matrix((data, (row, col)), shape=self.shape)


    def C_mat(self, ):
        diagonals = [[-1/2]*(self.N-2),  [1/2]*(self.N-2)]

        return csc_matrix(diags(diagonals, [-1, 1]))


    def D_mat(self, ):
        diagonals = [[-1]*(self.N-2),  [2]*(self.N-1), [-1]*(self.N-2)]

        return 1/self.H * csc_matrix(diags(diagonals, [-1, 0, 1]))


    def E1(self, t):
        
        consts = [Equation.dg_0(t), Equation.dg_1(t), Equation.g_0(t)**2, Equation.g_0(t)*Equation.g_1(t), \
                  Equation.g_1(t)*Equation.g_0(t), Equation.g_1(t)**2, Equation.h_0(t), Equation.h_1(t),\
                  -Equation.beta*Equation.g_0(t), -Equation.beta*Equation.g_1(t), Equation.beta*Equation.g_1(t), -Equation.beta*Equation.g_0(t)]  
        
        row = np.array([0, 0, 0, 0])
        col = np.array([0, 1, self.N - 1-2, self.N - 1-1])
        data = [self.H * np.array([1/6, 0, 0, 0])]
        data += [self.H * np.array([0, 0, 0, 1/6])]
        data += [np.array([-1/6, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 1/6])]
        data += [np.array([ -1/2, 0, 0, 0])]
        data += [np.array([ 0, 0, 0, 1/2])]
        data += [1/self.H * np.array([ -1, 0, 0, 0])]
        data += [1/self.H * np.array([0, 0, 0, -1])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data = np.array(sum(map(lambda x: consts[x]*data[x] , range(len(consts))))).ravel()
        return  csc_matrix((data, (row, col)), shape=(1, self.shape[0])).toarray().ravel()

    def E2(self, t):

        consts = [
            Equation.dh_0(t), 
            Equation.dh_1(t), 
            Equation.g_0(t)*Equation.h_0(t), 
            Equation.g_0(t)*Equation.h_1(t), 
            Equation.g_1(t)*Equation.h_0(t),
            Equation.g_1(t)*Equation.h_1(t),
            Equation.h_0(t)*Equation.g_0(t),
            Equation.h_0(t)*Equation.g_1(t), 
            Equation.h_1(t)*Equation.g_0(t),
            Equation.h_1(t)*Equation.g_1(t),
            Equation.beta*Equation.h_0(t),
            Equation.beta*Equation.h_1(t), 
            -Equation.beta*Equation.h_1(t),
            Equation.beta*Equation.h_0(t)
        ]

        row = np.array([0, 0, 0, 0])
        col = np.array([0, 1, self.N - 1-2, self.N - 1-1])
        data = [self.H * np.array([1/6, 0, 0, 0])]
        data += [self.H * np.array([0, 0, 0, 1/6])]
        data += [np.array([-1/6, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 1/6])]
        data += [np.array([ -1/6, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])] 
        data += [np.array([ 0, 0, 0, 1/6])]
        data += [1/self.H * np.array([ -1, 0, 0, 0])]
        data += [1/self.H * np.array([0, 0, 0, -1])]
        data += [np.array([0, 0, 0, 0])]
        data += [np.array([0, 0, 0, 0])]
        data = np.array(sum(map(lambda x: consts[x]*data[x] , range(len(consts))))).ravel()
        
        return  csc_matrix((data, (row, col)), shape=(1, self.shape[0])).toarray().ravel()

    def derivative(self, t, y):
        
        assert y.shape == (2*(self.N - 1), 1)                             
        u, v = np.split(y.flatten(), 2)
        gamma = -1 * self.A_inv.dot(self.B(u, u) + (Equation.g_0(t)*(self.B_top + self.B_left) + \
                Equation.g_1(t)*(self.B_bottom+self.B_right) - Equation.beta*self.D).dot(u)+ self.C.dot(v) + self.E1(t))  
        
        delta = -1 * self.A_inv.dot(self.B(v, u) + self.B(u, v) + (self.B_top + self.B_left).dot(Equation.h_0(t)*u + Equation.g_0(t)*v) + \
                  (self.B_right + self.B_bottom).dot(Equation.h_1(t)*u + Equation.g_1(t)*v) + Equation.beta*self.D.dot(v) + self.E2(t))
        return np.concatenate([gamma.ravel(), delta.ravel()]).reshape(-1,)                  

    def algo_1(self, ):
        self.algo = solve_ivp(fun=self.derivative,t_span=(0, Equation.end_time), \
                              y0=np.concatenate([np.array([Equation.u0(i) for i in self.domain]), \
                   np.array([Equation.v0(i) for i in self.domain])]), \
                      t_eval= np.array([Equation.end_time]), vectorized=True)
        
        self.u_t, self.v_t =  np.split(self.algo.y.ravel(), 2)
        return (self.u_t, self.v_t)

In [ ]:
eqn = Equation([4])